First `import` the dependencies. We are using dask distributed client to run the program on dask worker nodes. Edit the `dask_client.py` file to change the options for the dask scheduler

In [10]:
#!/usr/bin/env python3

import dask.dataframe as df
import dask_client as dc

Next we read the bigquery export from the json file.

In [46]:
def read_json_as_dataframe(path):
    return df.read_json(path, blocksize=2**28)

dc.run_client()
json_data = read_json_as_dataframe('data/json/bq-results-20190611.json')

Now lets explore and see the data types and print the head

In [47]:
print(json_data.head())
print(json_data.dtypes)

                                            app_info  \
0  {'id': 'org.phidatalab.radar_armt', 'version':...   
1  {'id': 'org.phidatalab.radar_armt', 'version':...   
2  {'id': 'org.phidatalab.radar_armt', 'version':...   
3  {'id': 'org.phidatalab.radar_armt', 'version':...   
4  {'id': 'org.phidatalab.radar_armt', 'version':...   

                                              device  \
0  {'category': 'mobile', 'mobile_brand_name': 'S...   
1  {'category': 'mobile', 'mobile_brand_name': 'S...   
2  {'category': 'mobile', 'mobile_brand_name': 'S...   
3  {'category': 'mobile', 'mobile_brand_name': 'S...   
4  {'category': 'mobile', 'mobile_brand_name': 'S...   

                 event_name  \
0  notification_rescheduled   
1     prepared_kafka_object   
2     prepared_kafka_object   
3             send_to_cache   
4             send_to_cache   

                                        event_params  \
0  [{'key': 'firebase_screen_class', 'value': {'s...   
1  [{'key': 'firebase_scree

Print the column names

In [49]:
json_data.columns

Index(['app_info', 'device', 'event_name', 'event_params',
       'event_previous_timestamp', 'event_server_timestamp_offset',
       'event_timestamp', 'geo', 'user_first_touch_timestamp',
       'user_properties', 'user_pseudo_id'],
      dtype='object')

In [50]:
unique_users = json_data.user_pseudo_id.unique()

In [34]:
len(unique_users)

604

In [68]:
def sort_and_group(sort_by, group_by):
    return json_data.set_index('user_first_touch_timestamp').groupby('user_pseudo_id')

In [75]:
users = sort_and_group('user_first_touch_timestamp','user_pseudo_id')['event_timestamp']

In [76]:
count_per_user = users.count()

In [77]:
%matplotlib inline
import matplotlib.pyplot as plt